In [1]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/hydrological_forecasting/"

In [2]:
# IMPORTs
import sys, os
import glob
from dask import dataframe as dd

# to link the lib in py scripts as well
os.chdir( wdir )
sys.path.insert( 0, os.path.join(os.path.abspath(os.getcwd()),'lib') )

from lib import *

In [3]:
# SETUP
variables = ['precipitation','temperature']
basin = 'plan'

data_path = '/media/windows/projects/hydrological_forecasting/machine_learning/data/'

init_ref = '03'

In [4]:
for variable in variables:

    obs_dir = data_path + "observed/" + basin + "/" + variable + "/daily/obs/mean/"
    fct_dir = data_path + "observed/" + basin + "/" + variable + "/daily/fullfilled/mean/"

    fct_dir = fct_dir.format(
            basin=basin, variable=variable, output_type='mean')

    print(fct_dir)

    dirs = glob.glob( fct_dir + "*.csv" )

    output_path = "/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/" + basin + "/" + variable + "/biased_filled/mean/"
    mkNestedDir( os.path.dirname( output_path ) )

    dates = []

    for el in dirs:
        dates.append( os.path.basename( el[:-4] ) ) 

    is_first = True

    for date in dates:

        print( date )

        ## OBSERVED DATASET ##
        fileName_obs = obs_dir + date + ".csv"

        df_obs = pd.read_csv(fileName_obs, header=0, index_col=0, parse_dates=['datetime'], sep=';')
        df_obs = df_obs['values']
        if variable == 'precipitation':
            df_obs = df_obs.resample('h').sum()
        elif variable == 'temperature':
            df_obs = df_obs.resample('h').mean()

        ## FORECAST DATASET ##
        fileName_fct = fct_dir + date + ".csv"

        df_fct = pd.read_csv(fileName_fct, header=0, index_col=0, parse_dates=['datetime'], sep=';')
        # df_fct.index = df_fct.index.tz_localize('UTC')
        df_fct = df_fct['values']

        ################################ PRECIPITATION HOURLY #####################################
        plots = []

        plt_conf = {}
        plt_conf["label"] = 'Observed'
        plt_conf["color"] = '#e66101'
        plots.append( (df_obs, plt_conf) )

        plt_conf = {}
        plt_conf["label"] = 'ICON forecast'
        plt_conf["color"] = '#8078bc'
        plots.append( (df_fct, plt_conf) )

        import matplotlib.dates as mdates
        x_major_formatter=mdates.DateFormatter('%m-%dT%H')

        outfile = output_path + 'comparison/' + date + "." + output_format

        print(outfile)
        mkNestedDir( os.path.dirname( outfile ) )
        createPlot( plots, "Time $[hour]$", 'Prec. [$mm/hour$]', outfile, x_major_formatter=x_major_formatter, my_dpi=50, height=80 )

        outfile_hd = output_path + 'comparison/HD/' + date + "." + output_format
        mkNestedDir( os.path.dirname( outfile_hd ) )
        createPlot( plots, "Time $[hour]$", 'Prec. [$mm/hour$]', outfile_hd, x_major_formatter=x_major_formatter, my_dpi=600, height=80 )

        ################################ BOXPLOT BIAS ###############################################
        ############################ ICON forecast - observed ###########################################
        diff_fct_obs = df_fct - df_obs
        # diff_fct_obs.index = diff_fct_obs.index.hour

        # first_day_start = dt.datetime.strptime(date, '%Y%m%d')
        # first_day_end = first_day_start + dt.timedelta( hours=24 )

        # diff_fct_obs = df_fct[ first_day_start:first_day_end ] - df_obs[ first_day_start:first_day_end ]

        if is_first == True:
            diff_df = pd.DataFrame( diff_fct_obs, columns=['values'] )
            diff_df = diff_df.reset_index().drop(columns='datetime')
            diff_df = diff_df.rename( columns={'values': date} )
            is_first = False
        else:
            diff_df[ date ] = diff_fct_obs

        myFig = plt.figure()
        bp = diff_df.T.boxplot( grid=False )

        outfile = output_path + 'comparison/boxplot/' + date + "." + output_format
        mkNestedDir( os.path.dirname(outfile) )
        myFig.savefig(outfile, format=output_format)

        plt.close()

/media/windows/projects/hydrological_forecasting/machine_learning/data/observed/plan/precipitation/daily/fullfilled/mean/
20210615
/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/plan/precipitation/biased_filled/mean/comparison/20210615.png
20210616
/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/plan/precipitation/biased_filled/mean/comparison/20210616.png
20210617
/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/plan/precipitation/biased_filled/mean/comparison/20210617.png
20210618
/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/plan/precipitation/biased_filled/mean/comparison/20210618.png
20210619
/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/postprocess/plan/precipitation/biased_filled/mean/compari